In [10]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.exceptions import SpotifyException
import matplotlib.pyplot as plt
import random

print("All libraries imported successfully.")

All libraries imported successfully.


In [11]:
# Load the .env file variables
load_dotenv("../.env")

True

In [12]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")

In [13]:
# Authenticate with Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [14]:
# Helper functions
def get_available_genres():
    """Retrieve available genre seeds from Spotify."""
    return sp.recommendation_genre_seeds()['genres']

def search_artist_id(artist_name):
    """Search for an artist and return their Spotify ID."""
    result = sp.search(q='artist:' + artist_name, type='artist', limit=1)
    if result['artists']['items']:
        return result['artists']['items'][0]['id']
    print(f"Warning: No artist found for '{artist_name}'")
    return None

def search_track_id(track_name):
    """Search for a track and return its Spotify ID."""
    result = sp.search(q='track:' + track_name, type='track', limit=1)
    if result['tracks']['items']:
        return result['tracks']['items'][0]['id']
    print(f"Warning: No track found for '{track_name}'")
    return None

def get_combined_recommendations(genres=None, artists=None, songs=None, mood=None, limit=10):
    """Fetch combined recommendations based on selected genres, artists, songs, and mood."""
    seeds = {'seed_genres': [], 'seed_artists': [], 'seed_tracks': []}
    
    # Validate and add genres
    available_genres = get_available_genres()
    valid_genres = [genre.lower() for genre in genres if genre.lower() in available_genres]
    if not valid_genres and genres:
        print("Error: None of the provided genres are valid.")
    seeds['seed_genres'].extend(valid_genres)
    
    # Validate and add artist IDs
    artist_ids = [search_artist_id(artist) for artist in artists]
    artist_ids = [artist_id for artist_id in artist_ids if artist_id]
    if not artist_ids and artists:
        print("Error: None of the provided artists were found.")
    seeds['seed_artists'].extend(artist_ids)
    
    # Validate and add track IDs
    track_ids = [search_track_id(song) for song in songs]
    track_ids = [track_id for track_id in track_ids if track_id]
    if not track_ids and songs:
        print("Error: None of the provided songs were found.")
    seeds['seed_tracks'].extend(track_ids)

    # Check for at least one valid seed
    if not (seeds['seed_genres'] or seeds['seed_artists'] or seeds['seed_tracks']):
        print("Error: At least one valid genre, artist, or song is required for recommendations.")
        return []

    # Define mood-based parameters
    mood_map = {
        'happy': {'target_valence': 0.8, 'target_danceability': 0.7},
        'sad': {'target_valence': 0.2, 'target_acousticness': 0.7},
        'dance': {'target_danceability': 0.9, 'target_energy': 0.8},
        'relaxed': {'target_valence': 0.6, 'target_acousticness': 0.8, 'target_energy': 0.3, 'target_tempo': 80.0},
        'energetic': {'target_energy': 0.9, 'target_danceability': 0.8, 'target_tempo': 140.0},
        'romantic': {'target_valence': 0.7, 'target_acousticness': 0.6, 'target_energy': 0.4},
        'calm': {'target_valence': 0.4, 'target_acousticness': 0.9, 'target_energy': 0.2, 'target_tempo': 60.0},
        'angry': {'target_energy': 0.8, 'target_valence': 0.2, 'target_danceability': 0.5, 'target_tempo': 150.0},
        'focus': {'target_acousticness': 0.6, 'target_instrumentalness': 0.5, 'target_tempo': 100.0},
        'uplifting': {'target_valence': 0.9, 'target_energy': 0.7, 'target_danceability': 0.6, 'target_tempo': 120.0}
    }

    # Apply mood-based parameters if valid mood is provided
    mood_params = mood_map.get(mood.lower(), {}) if mood else {}

    try:
        # Fetch recommendations with mood filters if applicable
        recommendations = sp.recommendations(seed_genres=seeds['seed_genres'], 
                                             seed_artists=seeds['seed_artists'], 
                                             seed_tracks=seeds['seed_tracks'], 
                                             limit=limit, 
                                             **mood_params)
        # Return formatted track recommendations
        return [(track['name'], track['artists'][0]['name']) for track in recommendations['tracks']]
    except SpotifyException as e:
        print(f"Spotify API error: {e}")
        return []


In [15]:
def input_genres():
    """Prompt user for genres and validate selection."""
    available_genres = get_available_genres()
    print("Available genres:", available_genres)
    genres = input("Enter genres separated by commas (or press Enter to skip): ").lower().split(',')
    genres = [genre.strip() for genre in genres if genre.strip() in available_genres]
    if genres:
        print(f"Genres selected: {genres}")
    else:
        print("No valid genres selected.")
    return genres

def input_artists():
    """Prompt user for artist names and validate selection."""
    artists = input("Enter artist names separated by commas (or press Enter to skip): ").split(',')
    artists = [artist.strip() for artist in artists if artist.strip()]
    artist_ids = []
    for artist in artists:
        artist_id = search_artist_id(artist)
        if artist_id:
            artist_ids.append(artist)
    if artist_ids:
        print(f"Artists selected: {artist_ids}")
    else:
        print("No valid artists found.")
    return artist_ids

def input_songs():
    """Prompt user for song names and validate selection."""
    songs = input("Enter song names separated by commas (or press Enter to skip): ").split(',')
    songs = [song.strip() for song in songs if song.strip()]
    track_ids = []
    for song in songs:
        track_id = search_track_id(song)
        if track_id:
            track_ids.append(song)
    if track_ids:
        print(f"Songs selected: {track_ids}")
    else:
        print("No valid songs found.")
    return track_ids    

def input_mood():
    """Prompt user for mood and validate selection."""
    available_moods = ["happy", "sad", "dance", "relaxed", "energetic", "romantic", "calm", "angry", "focus", "uplifting"]
    print("Available moods:", ", ".join(available_moods))
    mood = input("Enter a mood (or press Enter to skip): ").strip().lower()
    if mood in available_moods:
        print(f"Mood selected: {mood}")
        return mood
    elif mood:
        print("Invalid mood selected. Skipping mood filter.")
    return None


In [16]:
def main():
    print("Welcome to the Spotify Recommendation System!")
    
    # Collect genres, artists, songs, and mood from user input
    genres = input_genres()
    artists = input_artists()
    songs = input_songs()
    mood = input_mood()
    
    # Fetch and display recommendations
    recommendations = get_combined_recommendations(genres=genres, artists=artists, songs=songs, mood=mood, limit=10)
    if recommendations:
        print("\nRecommended Songs:")
        for idx, (track_name, artist_name) in enumerate(recommendations, start=1):
            print(f"{idx}. {track_name} by {artist_name}")
    else:
        print("No recommendations available. Try using different input options.")


In [17]:
# Run the program
if __name__ == "__main__":
    main()

Welcome to the Spotify Recommendation System!
Available genres: ['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philipp